# Board Game Data Model EDA

## Introduction
This notebook provides exploration of the data used for the Board Game Data Model and the resulting data manipulations to create the data pipeline required to create the model. The final data pipelines can be found in `create_tables.py` which are derived from this notebook.

## Data Source
The data used in this project is from the two sources noted below. The data is available at `/data/` and data dictionaries for data used in the Board Game Data Model can be found at `/data/README.md`.

**[BoardGameGeek (BGG) API](https://boardgamegeek.com/wiki/page/BGG_XML_API)** 
The API provides XML output which is then compiled into csv format. Endpoints accessed include:
- `https://boardgamegeek.com/xmlapi/boardgame/` provides information about a particular boardgame - saved to `/data/raw/bgg/games.csv`.
- `https://boardgamegeek.com/xmlapi/geeklist/` which provides GeekList information - saved to `/data/raw/bgg/lists.csv`.

**[Board Game Atlas](https://www.boardgameatlas.com/)**
The API provides JSON output which is compiled into JSON format. Endpoints accessed include:
- `https://www.boardgameatlas.com/api/docs/prices` provides price data for a game - saved to `/data/raw/atlas/prices/`.
- `https://www.boardgameatlas.com/api/docs/users` provides a list of all Board Game Atlas users with information - saved to `/data/raw/atlas/json_user.json`.
- `https://www.boardgameatlas.com/api/docs/reviews` provides game reviews as well as additional information on the user that made the review - saved to `/data/raw/atlas/reviews/` and `/data/raw/atlas/user/`, respectively.
- `https://www.boardgameatlas.com/api/docs/search` provides board game details - saved to `/data/raw/atlas/games/`


## Setup

In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat_ws, to_timestamp
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, weekofyear, dayofweek, monotonically_increasing_id
from pyspark.sql.types import IntegerType

config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [2]:
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

In [3]:
input_data = "data/raw"
output_data = "data/output"

## EDA and Data Pipelines

### BGGDetails Table

The dimension table corresponding to the `https://boardgamegeek.com/xmlapi/boardgame/`which provides information about a particular boardgame - saved to `/data/raw/bgg/games.csv`.

In [4]:
# get filepath and read data
bgg_games_data = os.path.join(input_data, "bgg/games.csv")
bgg_games_df = spark.read.csv(bgg_games_data, header=True, inferSchema=True)

In [5]:
bgg_games_df.printSchema()

root
 |-- bgg_id: integer (nullable = true)
 |-- maxplayers: double (nullable = true)
 |-- maxplaytime: double (nullable = true)
 |-- age: double (nullable = true)
 |-- minplayers: double (nullable = true)
 |-- minplaytime: double (nullable = true)
 |-- name: string (nullable = true)
 |-- year_published: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- category: string (nullable = true)
 |-- compilation: string (nullable = true)
 |-- designer: string (nullable = true)
 |-- family: string (nullable = true)
 |-- mechanic: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- users_rated: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- datetime_extracted: string (nullable = true)



In [6]:
# convert column types
bgg_games_df = bgg_games_df.withColumn("datetime_extracted", 
                                       to_timestamp(col("datetime_extracted"),format="dd/MM/yyyy HH:mm:ss")
                                      ).withColumn("year_published", col("year_published").cast(IntegerType()))

In [7]:
# rename columns to match BGGDetails Table
bgg_games_df = bgg_games_df.withColumnRenamed("bgg_id", "bgg_game_id"
                                    ).withColumnRenamed("maxplayers", "max_players"
                                    ).withColumnRenamed("maxplaytime", "max_playtime"
                                    ).withColumnRenamed("age", "min_age"
                                    ).withColumnRenamed("minplayers", "min_players"
                                    ).withColumnRenamed("minplaytime", "min_playtime"
                                    ).withColumnRenamed("name", "game_name"
                                    ).withColumnRenamed("users_rated", "num_ratings")

In [8]:
# drop PK duplicates and missing values
bgg_games_df = bgg_games_df.dropDuplicates(["bgg_game_id"]).dropna(subset=["bgg_game_id"])

In [9]:
bgg_games_df.limit(5).toPandas()

,bgg_game_id,max_players,max_playtime,min_age,min_players,min_playtime,game_name,year_published,artist,category,compilation,designer,family,mechanic,publisher,num_ratings,average_rating,datetime_extracted
0,148,6.0,60.0,8.0,3.0,60.0,Die Glücksritter,1999,Nicholas Price,"Bluffing,Medieval",None,Klaus Kreowski,None,"Rock-Paper-Scissors,Simultaneous Action Selection",Schmidt Spiele,136,5.70074,2022-11-29 15:12:16
1,463,2.0,20.0,13.0,2.0,20.0,Magic: The Gathering,1993,"""Victor Adame,Miao Aili,Randis Albion,Rob Alex...",Brian Durfee,Jeff Easley,Cole Eastburn,Robert Eggleton,Jesper Ejsing,Randy Elliott,Steve Ellis (I),Larry Elmore,NaT
2,471,4.0,90.0,10.0,2.0,90.0,Totem,1995,François Bruel,Civilization,None,"Philippe des Pallières,Patrice Pillet",None,None,Queen Games,121,5.4876,2022-11-29 15:12:16
3,496,5.0,30.0,10.0,3.0,30.0,Dilemma,2000,Paul Dawiz,"Action / Dexterity,Card Game",None,Alex Randolph,None,None,"F.X. Schmid,Oliphante,OPEN'N PLAY",31,4.32581,2022-11-29 15:12:16
4,833,2.0,360.0,12.0,2.0,360.0,For the People,1998,"Rodger B. MacGowan,Kurt Miller,Mark Simonitch","American Civil War,Civil War,Post-Napoleonic,W...",None,Mark Herman,"Country: USA,Players: Two Player Only Games","Campaign / Battle Card Driven,Dice Rolling,Poi...","The Avalon Hill Game Co,Devir,GMT Games",1621,7.78134,2022-11-29 15:12:16


In [11]:
bgg_games_df.describe().toPandas()

,summary,bgg_game_id,max_players,max_playtime,min_age,min_players,min_playtime,game_name,year_published,artist,category,compilation,designer,family,mechanic,publisher,num_ratings,average_rating
0,count,342570,140799,140799,140799,140799,140799,342570,140785,62525,138529,5755,117801,101677,120903,147390,342549,342562
1,mean,188762.9761566979,6.411863720623016,858.6511551928636,7.912776369150349,1.9429612426224618,42.850510301919755,Infinity,1858.8867990197818,1380.8,2021.0,1951.0,170.675,None,None,42.0,72.37918968100736,3.298406689257453
2,stddev,108539.46528272027,43.89174060587652,190916.60207645257,7.345503281725199,0.8390772539365494,490.61854217412287,NaN,522.6926523412053,909.6278045756642,NaN,NaN,179.46259359543424,None,None,NaN,1036.2335970509614,25.438103701421433
3,min,1,0.0,0.0,0.0,0.0,0.0,! That Bastard Is Trying To Steal Our Gold !,-3500,"1616-1633""","Bienchen summ herum""""""","""""""The Naturalist"""" Scenario (fan expansion fo...",3000-500 BC,"""Donald """"DJ"""" Carr",100BC-700AD,"""""""Let's Talk"""" Company""",500-100BC,"(Public Domain),A. N. Myers & Co,ABRA,Adolf Sa..."
4,max,375608,11299.0,6.3072E7,2017.0,50.0,157680.0,～幻想郷TRPG～,2026,齋藤 海有 (Wataru Saitou),"Łukasz Witusiński""",파워그리드 재충전 확장: 지도 모음집 (Power Grid Recharged – M...,"ｱｷﾋﾛｲﾄｵ (Akihiro Itoh),Kwaji,Daichi Okano,Kito...","Word Games: Spelling / Letters,Players: Two Pl...",Zone of Control,＠BUDAcafe (アットブダカフェ),von Braus publishing,Winning Eleven Productions


#### Summary of Data

The details table above indicates that:
- There are 342k rows of data
- All rows have the bgg_id and game_name (and num_ratings and average_ratings are almost complete)
- Approximately 50% have details such as max/min playtime, max/min players, age, and published year
- All other columns with game descriptions (such as mechanic, artist, etc) have varying amounts of completeness
- Looking at the min/max values for various columns, it's clear that there are some erroneous data entered (negative value as the min for published year) - it's expected that some form of data cleaning would be required after pulling from the data model.

In [12]:
bgg_games_df.write.parquet(os.path.join(output_data, "bgg_games.parquet"), partitionBy=["year_published"], mode="overwrite")

### AtlasUsers Table

The dimension table corresponding to user data from the following sources:
- `https://www.boardgameatlas.com/api/docs/users` provides a list of all Board Game Atlas users with information - saved to `/data/raw/atlas/json_user.json`.
- `https://www.boardgameatlas.com/api/docs/reviews` provides game reviews as well as additional information on the user that made the review - saved to `/data/raw/atlas/reviews/` and `/data/raw/atlas/user/`, respectively.

In [13]:
# get filepath and read data for the first data source
atlas_users1_data = os.path.join(input_data, "atlas/json_user.json")
atlas_users1_df = spark.read.json(atlas_users1_data, multiLine=True)

In [14]:
atlas_users1_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experience: long (nullable = true)
 |-- gold: long (nullable = true)
 |-- images: struct (nullable = true)
 |    |-- large: string (nullable = true)
 |    |-- medium: string (nullable = true)
 |    |-- original: string (nullable = true)
 |    |-- small: string (nullable = true)
 |    |-- thumb: string (nullable = true)
 |-- is_new: boolean (nullable = true)
 |-- is_supporter: boolean (nullable = true)
 |-- level: long (nullable = true)
 |-- level_curr_exp: long (nullable = true)
 |-- level_next_exp: long (nullable = true)
 |-- level_percent: double (nullable = true)
 |-- ringCss: string (nullable = true)
 |-- url: string (nullable = true)
 |-- username: string (nullable = true)



In [15]:
# select columns of interest and rename columns to match Table
atlas_users1_df = atlas_users1_df.select(col("username").alias("user_name"),
                                         "url", "description",
                                         col("gold").alias("atlas_gold"),
                                         col("experience").alias("atlas_exp"),
                                         col("level").alias("atlas_level"))

In [16]:
# drop duplicates and missing values corresponding with the table PK
atlas_users1_df = atlas_users1_df.dropDuplicates(["user_name"]).dropna(subset=["user_name"])

In [17]:
atlas_users1_df.limit(5).toPandas()

,user_name,url,description,atlas_gold,atlas_exp,atlas_level
0,Azmaer,https://www.boardgameatlas.com/u/Azmaer,,1423,373,32
1,BlueGiant42,https://www.boardgameatlas.com/u/BlueGiant42,,191,191,25
2,Brolands,https://www.boardgameatlas.com/u/Brolands,,37,37,8
3,Carptarts,https://www.boardgameatlas.com/u/Carptarts,,68,68,14
4,Eleanor10,https://www.boardgameatlas.com/u/Eleanor10,"Hi, I am Eleanor Mila. I have been working as ...",0,0,0


In [18]:
atlas_users1_df.describe().toPandas()

,summary,user_name,url,description,atlas_gold,atlas_exp,atlas_level
0,count,5423,5423,5423,5423,5423,5423
1,mean,7734.0,None,11111.0,145.96109164669002,142.65941360870366,7.328047206343353
2,stddev,NaN,None,NaN,1605.87560991176,1647.67635507376,11.485363514849013
3,min,-Rinser-,https://www.boardgameatlas.com/u/-Rinser-,,0,0,0
4,max,انا,https://www.boardgameatlas.com/u/انا,—> This is the reduced form of the correspond...,75689,75559,88


The data shows there are 5423 users and all columns are complete.

In [29]:
# get filepath and read data for the second data source
atlas_users2_data = os.path.join(input_data, "atlas/users/*.json")
atlas_users2_df = spark.read.json(atlas_users2_data, multiLine=True)

In [30]:
atlas_users2_df.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- created_at_ago: string (nullable = true)
 |-- forum_email: boolean (nullable = true)
 |-- id: string (nullable = true)
 |-- imageUrl: string (nullable = true)
 |-- images: struct (nullable = true)
 |    |-- large: string (nullable = true)
 |    |-- medium: string (nullable = true)
 |    |-- original: string (nullable = true)
 |    |-- small: string (nullable = true)
 |    |-- thumb: string (nullable = true)
 |-- isOwner: boolean (nullable = true)
 |-- isVerified: boolean (nullable = true)
 |-- is_moderator: boolean (nullable = true)
 |-- is_new: boolean (nullable = true)
 |-- is_owner: boolean (nullable = true)
 |-- is_partner: boolean (nullable = true)
 |-- is_premium: boolean (nullable = true)
 |-- message_email: boolean (nullable = true)
 |-- num_followers: long (nullable = true)
 |-- pfUser: struct (nullable = true)
 |    |-- objectId: string (nullable = true)
 |-- price_email: boolean (nullable = true)
 |-- updated_at: string (nu

In [31]:
# only select columns of interest
atlas_users2_df = atlas_users2_df.select(col("id").alias("atlas_user_id"),
                                         col("is_premium").alias("atlas_premium"),
                                         col("is_partner").alias("atlas_partner"),
                                         col("is_moderator").alias("atlas_moderator"),
                                         col("num_followers").alias("atlas_followers"),
                                         col("username").alias("user_name"))

In [32]:
# drop duplicates and missing values for Table PK
atlas_users2_df = atlas_users2_df.dropDuplicates(["atlas_user_id"]).dropna(subset=["atlas_user_id"])

In [33]:
atlas_users2_df.limit(5).toPandas()

,atlas_user_id,atlas_premium,atlas_partner,atlas_moderator,atlas_followers,user_name
0,3TWDgLXYfn,False,False,False,0,Rynn
1,3aQKPZmQVr,False,False,False,0,beardgoggles
2,9FzchNX38c,False,False,False,0,LKTrashmouth
3,BXzlhlqwCb,False,False,False,0,guzforster
4,JdGjsmHST0,False,False,False,0,Xist


In [34]:
atlas_users2_df.describe().toPandas()

,summary,atlas_user_id,atlas_followers,user_name
0,count,1880,1880,1880
1,mean,None,0.0,7734.0
2,stddev,None,0.0,NaN
3,min,03pnbGwpn7,0,07734
4,max,zwJQyVMADi,0,zxcvbn


The table indicates that only 1880 users are in this table - this would correspond to users that made reviews as that is how this information was extracted. It also indicates that while `atlas_followers` was provided as a field in the data, it clearly isn't populated - as such it should be dropped.

In [35]:
# drop atlas followers
atlas_users2_df = atlas_users2_df.drop("atlas_followers")

In [38]:
# The two tables need to be joined based on the user ID
atlas_users_final_df = atlas_users1_df.join(atlas_users2_df, "user_name", how="left")

In [39]:
atlas_users_final_df.describe().toPandas()

,summary,user_name,url,description,atlas_gold,atlas_exp,atlas_level,atlas_user_id
0,count,5423,5423,5423,5423,5423,5423,1880
1,mean,7734.0,None,11111.0,145.96109164669002,142.65941360870366,7.328047206343353,None
2,stddev,NaN,None,NaN,1605.87560991176,1647.67635507376,11.485363514849013,None
3,min,-Rinser-,https://www.boardgameatlas.com/u/-Rinser-,,0,0,0,03pnbGwpn7
4,max,انا,https://www.boardgameatlas.com/u/انا,—> This is the reduced form of the correspond...,75689,75559,88,zwJQyVMADi


As <50% of the atlas_user_ids are available (only atlas_user_ids are available in the data for users that made reviews. Create a UID 'user_id' for the table.

In [41]:
atlas_users_final_df = atlas_users_final_df.withColumn("user_id", monotonically_increasing_id())

#### Summary of Data

The details table above indicates that:
- There are 5423 users with complete information corresponding to the first df
- 1880 users made reviews (second df) and there is additional information available for those users that have been added to the dataframe

In [42]:
atlas_users_final_df.write.parquet(os.path.join(output_data, "users.parquet"), partitionBy=["atlas_level"], mode="overwrite")

### AtlasDetails Table

The dimension table corresponding to `https://www.boardgameatlas.com/api/docs/search` which provides board game details - saved to `/data/raw/atlas/games/`

In [43]:
# get filepath to read data
atlas_games_data = os.path.join(input_data, "atlas/games/*.json")
atlas_games_df = spark.read.json(atlas_games_data, multiLine=True)

In [44]:
atlas_games_df.printSchema()

root
 |-- active: boolean (nullable = true)
 |-- amazon_rank: long (nullable = true)
 |-- artists: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- availability_status: string (nullable = true)
 |-- average_learning_complexity: double (nullable = true)
 |-- average_strategy_complexity: double (nullable = true)
 |-- average_user_rating: double (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- comment_count: long (nullable = true)
 |-- commentary: string (nullable = true)
 |-- cs_rating: double (nullable = true)
 |-- datetime_extracted: string (nullable = true)
 |-- description: string (nullable = true)
 |-- description_preview: string (nullable = true)
 |-- designers: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- images: struct

In [45]:
# the data available for atlas games is extensive with nested json
# only extract the columns of interest for the data model table and rename columns to match
atlas_games_df = atlas_games_df.select(col("id").alias("atlas_game_id"),
                                       col("name").alias("game_name"),
                                       "url", "datetime_extracted", "year_published",
                                       "min_players", "max_players", "min_playtime", "max_playtime",
                                       "min_age",
                                       col("primary_publisher.name").alias("primary_publisher"),
                                       col("primary_designer.name").alias("primary_designer"), 
                                       "artists",
                                       col("num_user_ratings").alias("num_ratings"),
                                       col("average_user_rating").alias("average_rating"),
                                       col("num_user_complexity_votes").alias("num_complexity"),
                                       "average_learning_complexity", "average_strategy_complexity"
                                      )

In [46]:
# drop duplicate and missing values associated with Table PK
atlas_games_df = atlas_games_df.dropDuplicates(["atlas_game_id"]).dropna(subset=["atlas_game_id"])

In [47]:
# convert list to string format and convert string date to timstamp
atlas_games_df = atlas_games_df.withColumn("artists", concat_ws(",", "artists")
                    ).withColumn("datetime_extracted", to_timestamp(col("datetime_extracted"),format="dd/MM/yyyy HH:mm:ss"))

In [48]:
atlas_games_df.limit(5).toPandas()

,atlas_game_id,game_name,url,datetime_extracted,year_published,min_players,max_players,min_playtime,max_playtime,min_age,primary_publisher,primary_designer,artists,num_ratings,average_rating,num_complexity,average_learning_complexity,average_strategy_complexity
0,2Acdazgmgl,Genesis,https://www.boardgameatlas.com/en-CA/game/2Acd...,2022-12-02 11:05:31,2010,2,6,60,60,10,Gigantoskop,Peter Hansson,Erik Lundström,7,2.142857,0,0.0,0.0
1,2GAg97QP3P,Boss Monster: The Golden Dragon,https://www.boardgameatlas.com/en-CA/game/2GAg...,2022-12-02 11:05:31,2013,2,4,20,20,13,Brotherwise Games,Johnny O'Neal,,2,3.000000,0,0.0,0.0
2,6SwRxlJXHN,Trivial Pursuit X,https://www.boardgameatlas.com/en-CA/game/6SwR...,2022-12-02 11:05:31,2017,4,99,30,90,18,Hasbro,None,,1,0.000000,0,0.0,0.0
3,6maEIahmuS,Founding Fathers: Ladies & Orators,https://www.boardgameatlas.com/en-CA/game/6maE...,2022-12-02 11:05:31,2017,3,6,120,300,10,Up and Away Games,Rick Heli,Luca Cammisa,1,3.500000,0,0.0,0.0
4,6nJLBBvRMY,Cuisine a la Card,https://www.boardgameatlas.com/en-CA/game/6nJL...,2022-12-02 11:05:31,2015,2,4,15,45,13,InMotion,Dani Demarest,"Dani Demarest,Sarah Drake",2,3.000000,0,0.0,0.0


In [49]:
atlas_games_df.describe().toPandas()

,summary,atlas_game_id,game_name,url,year_published,min_players,max_players,min_playtime,max_playtime,min_age,primary_publisher,primary_designer,artists,num_ratings,average_rating,num_complexity,average_learning_complexity,average_strategy_complexity
0,count,23500,23500,23500,21799,22220,22220,21344,21344,21025,21316,18701,23500,23500,23500,23500,23500,23500
1,mean,None,Infinity,None,2007.5034175879628,1.954950495049505,5.685688568856886,60.615817091454275,1498.6169883808095,10.493697978596908,None,None,None,10.986382978723404,3.265780031155468,0.06876595744680851,0.09862385620220415,0.10939929285705254
2,stddev,None,NaN,None,45.045947882720775,0.7166660439362199,13.199158298220713,446.91375206011537,205346.2930730879,3.1749726968690557,None,None,None,38.9148257074197,0.7851720248523404,0.46334073200174203,0.5047019610367234,0.549138484531021
3,min,00AI3sb84B,"""La Garde recule!""",https://www.boardgameatlas.com/en-CA/game/00AI...,100,0,0,0,0,0,Black Monk,,,0,0.0,0,0.0,0.0
4,max,zzmsZaTDjT,🎰 Vegas Slot Machine 1973 Card Game,https://www.boardgameatlas.com/en-CA/game/zzms...,3500,10,999,60000,30000000,45,广州创游,西村裕 (Hiroshi Nishimura),長谷川 登鯉 (Tori Hasegawa),968,5.0,17,5.0,5.0


#### Summary of Data

The details table above indicates that:
- There are 23.5k rows of data
- All rows have the atlas_game_id, game_name, url, artists, and various rating columns
- All other columns are +80% complete
- The min/max values indicate that there are erroneous entered values such as minimum max_players as 0 - it is expected that some data cleaning would be required when pulling from the data model.

It is interesting that this is an order of magnitude lower than the 324k of BGG games data. This is potentially as Board Game Atlas is a newer site and appears to focus on popular board games whereas BGG allows users to upload any board game.

In [50]:
atlas_games_df.write.parquet(os.path.join(output_data, "atlas_games.parquet"), partitionBy=["year_published"], mode="overwrite")

### AtlasReviews Table

The fact table corresponding to `https://www.boardgameatlas.com/api/docs/reviews` provides game reviews - saved to `/data/raw/atlas/reviews/`.

In [69]:
# get filepath and read data
atlas_reviews_data = os.path.join(input_data, "atlas/reviews/*.json")
atlas_reviews_df = spark.read.json(atlas_reviews_data, multiLine=True)

In [70]:
atlas_reviews_df.printSchema()

root
 |-- createdAt: string (nullable = true)
 |-- date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)



In [71]:
# select columns of interest and rename corresponding with the data model
atlas_reviews_df = atlas_reviews_df.select(col("id").alias("review_id"),
                                           col("createdAt").alias("review_datetime"),
                                           col("user_id").alias("atlas_user_id"),
                                           col("game_id").alias("atlas_game_id"),
                                           "rating", "description")

In [72]:
# drop duplicate and missing rows corresponding with table PK and date
atlas_reviews_df = atlas_reviews_df.dropDuplicates(["review_id"]).dropna(subset=["review_id", "review_datetime"])

In [73]:
# convert string date to timestamp and add year and month columns for data partitioning for Parquet files               
atlas_reviews_df = atlas_reviews_df.withColumn("review_datetime", to_timestamp(col("review_datetime"),format="yyyy-MM-dd'T'HH:mm:ss.SSS'Z'")
                            ).withColumn("year", year("review_datetime")
                            ).withColumn("month", month("review_datetime"))

In [74]:
atlas_reviews_df.limit(5).toPandas()

,review_id,review_datetime,atlas_user_id,atlas_game_id,rating,description,year,month
0,01joCJZ1yh,2020-12-21 11:04:57,J99lC5QsBy,FYDls3xiob,4.00,None,2020,12
1,02QoWBeXz4,2019-02-15 01:06:04,N6Xb3ES5d2,IjMcGwdT95,4.00,None,2019,2
2,02iv770JRs,2020-01-28 20:51:48,uWAd7nP8Kj,qnheLzWlKy,3.50,None,2020,1
3,073m7s7XuT,2022-02-17 13:33:19,Db8v0NdFfV,9iBOPn3lES,4.00,None,2022,2
4,0AdiuuBpLF,2020-02-12 06:48:58,wAFb0Vtn4f,RW0HAoUctF,4.25,None,2020,2


In [75]:
atlas_reviews_df.describe().toPandas()

,summary,review_id,atlas_user_id,atlas_game_id,rating,description,year,month
0,count,226871,226871,226871,226871,56244,226871,226871
1,mean,None,None,None,3.5826648183328853,2025.9931077694237,2020.1637097733956,5.249238554068171
2,stddev,None,None,None,0.8343882517801763,6136.261669160499,1.019712819459849,4.099250123035813
3,min,0013QK6PYB,03pnbGwpn7,00AI3sb84B,0.0,,2018,1
4,max,zzxyt2S2Bt,zwJQyVMADi,zzmsZaTDjT,5.0,⊑⟒⌰⌰⍜ ⏃⋏⎅ ⍙⟒⌰☊⍜⋔⟒ ⏁⍜ ⋔⊬ ⏚⌰⍜☌\nNow that I've go...,2022,12


The data indicates:
- there are 227k reviews which all have an atlas_user_id, atlas_game_id, and rating
- The description is only available for approximatey 20% of the rows

Additional data needs to be added to the table to allow the BoardGameGeek game ID to be added to the fact table such that board game details provided by BoardGameGeek can be joined with the fact table.

In [76]:
# add the game name to the table from AtlasDetails
atlas_game_name = atlas_games_df.select("atlas_game_id", "game_name")
atlas_reviews_df = atlas_reviews_df.join(atlas_game_name, "atlas_game_id", how='left')

In [77]:
# add the BoardGameGeek game ID to the table from BGGdetails
# note that it was found that some bgg_game_id have the same game name; therefore, need to drop duplicates
# otherwise the fact table will grow based on duplicate BGG game names
bgg_game_name = bgg_games_df.select("bgg_game_id", "game_name").dropDuplicates(["game_name"])
atlas_reviews_df = atlas_reviews_df.join(bgg_game_name, "game_name", how='left')

In [78]:
# replace the atlas_user_id with the AtlasUsers PK of user_id
atlas_user_name = atlas_users_final_df.select("user_id", "atlas_user_id")
atlas_reviews_df = atlas_reviews_df.join(atlas_user_name, "atlas_user_id", how="left")
atlas_reviews_df = atlas_reviews_df.drop("atlas_user_id")

In [79]:
atlas_reviews_df.limit(5).toPandas()

,game_name,atlas_game_id,review_id,review_datetime,rating,description,year,month,bgg_game_id,user_id
0,Iquazu,hi0aXgk2aZ,4aA08h2hzi,2020-04-07 03:42:24,3.75,,2020,4,NaN,77309411333
1,Akrotiri,alg1QfJagu,spITeQJtiM,2020-04-07 03:42:04,4.00,,2020,4,154458.0,77309411333
2,The Tea Dragon Society Card Game,lS86TDfmnW,FchBgsIFzF,2020-04-07 03:36:12,3.00,,2020,4,236709.0,77309411333
3,Sprawlopolis,uBmwnw5A4w,ENazsB5p5K,2020-04-07 03:40:31,4.00,,2020,4,251658.0,77309411333
4,Via Nebula,95u6OPX2ip,DuqzEA8jWp,2020-04-07 03:37:04,4.50,,2020,4,191231.0,77309411333


In [80]:
atlas_reviews_df.describe().toPandas()

,summary,game_name,atlas_game_id,review_id,rating,description,year,month,bgg_game_id,user_id
0,count,226625,226871,226871,226871,56244,226871,226871,194364,226794
1,mean,Infinity,None,None,3.5826648183328853,2025.9931077694237,2020.1637097733956,5.249238554068171,139374.0826902101,8.047748043938763E11
2,stddev,NaN,None,None,0.8343882517801762,6136.261669160499,1.0197128194599077,4.099250123035812,93446.39469608468,4.8647687516762286E11
3,min,"""La Garde recule!""",00AI3sb84B,0013QK6PYB,0.0,,2018,1,1,0
4,max,🎰 Vegas Slot Machine 1973 Card Game,zzmsZaTDjT,zzxyt2S2Bt,5.0,⊑⟒⌰⌰⍜ ⏃⋏⎅ ⍙⟒⌰☊⍜⋔⟒ ⏁⍜ ⋔⊬ ⏚⌰⍜☌\nNow that I've go...,2022,12,374736,1709396983821


#### Summary of Data

The reviews table above indicates that:
- There are ~227k reviews which all have an atlas_user_id, atlas_game_id, and rating
- Most atlas_game_id were able to retrieve a game_name (99.9%)
- bgg_id were able to be added to 86% of the rows based on joining with game_name
- The above mismatch in game name between bgg_id and the game_name from the atlas_game_id could be resolved with investigation and string parsing

Overall, the reviews fact table is able to join data from BoardGameGeek for the majority of games providing it with rich supporting dimensional game data from multiple sources.

In [81]:
atlas_reviews_df.write.parquet(os.path.join(output_data, "reviews.parquet"), partitionBy=["year", "month"], mode="overwrite")

### AtlasPrices Table

The fact table corresponding to `https://www.boardgameatlas.com/api/docs/prices` which provides price data for a game - saved to `/data/raw/atlas/prices/`.

In [82]:
# get filepath and read data
atlas_prices_data = os.path.join(input_data, "atlas/prices/*/*.json")
atlas_prices_df = spark.read.json(atlas_prices_data, multiLine=True)

In [83]:
atlas_prices_df.printSchema()

root
 |-- condition: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- free_shipping_at: long (nullable = true)
 |-- free_shipping_text: string (nullable = true)
 |-- id: string (nullable = true)
 |-- in_stock: boolean (nullable = true)
 |-- is_sponsor: boolean (nullable = true)
 |-- msrp: double (nullable = true)
 |-- name: string (nullable = true)
 |-- price: double (nullable = true)
 |-- price_category: string (nullable = true)
 |-- price_text: string (nullable = true)
 |-- reviewsIframeUrl: string (nullable = true)
 |-- shipping: double (nullable = true)
 |-- storeNameSearch: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- updated_at_ago: string (nullable = true)
 |-- url: string (nullable = true)
 |-- variant: string (nullable = true)



In [84]:
# extract columns of interest and rename corresponding to the data model
atlas_prices_df = atlas_prices_df.select(col("id").alias("atlas_price_id"),
                                         col("updated_at").alias("price_datetime"),
                                         col("name").alias("game_name"),
                                         "price", "currency", "msrp", "url", "store_name", 
                                         "country", "price_category")

In [85]:
# drop missing or duplicates rows associated with the timestamp or the table PK
atlas_prices_df = atlas_prices_df.dropDuplicates(["atlas_price_id"]).dropna(subset=["atlas_price_id", "price_datetime"])

In [86]:
# convert string date to timestamp and add year and month columns for data partioning for Parquet files
atlas_prices_df = atlas_prices_df.withColumn("price_datetime", to_timestamp(col("price_datetime"),format="yyyy-MM-dd'T'HH:mm:ss.SSS'Z'")
                            ).withColumn("year", year("price_datetime")
                            ).withColumn("month", month("price_datetime"))

Follow the same approach as the reviews table to add the atlas_game_id and bgg_id to the fact table based on the game_name.

In [87]:
atlas_game_name = atlas_game_name.dropDuplicates(["game_name"])
atlas_prices_df = atlas_prices_df.join(atlas_game_name, "game_name", how='left')
bgg_game_name = bgg_games_df.select("bgg_game_id", "game_name").dropDuplicates(["game_name"])
atlas_prices_df = atlas_prices_df.join(bgg_game_name, "game_name", how='left')

In [88]:
atlas_prices_df.limit(5).toPandas()

,game_name,atlas_price_id,price_datetime,price,currency,msrp,url,store_name,country,price_category,year,month,atlas_game_id,bgg_game_id
0,The Grimm Masquerade,02nRJDZdMX,2022-11-30 21:19:43,19.99,USD,25.00,https://www.boardgameatlas.com/api/price?clien...,Card Haus,US,us,2022,11,iJRipsPFO4,254513.0
1,Quodd Heroes (NM condition),0BbElagazD,2022-12-01 16:04:21,0.00,USD,100.00,https://www.boardgameatlas.com/api/price?clien...,Noble Knight Games,US,used,2022,12,None,NaN
2,Colors of Paris,0DDH6l1StJ,2022-11-28 11:15:02,39.99,USD,49.99,https://www.boardgameatlas.com/api/price?clien...,Boardlandia,US,us,2022,11,MDx0rG1VlF,269160.0
3,Carcassonne: Big Box (2022) (PREORDER),0QOSK8DLSm,2022-12-02 21:01:50,134.95,AUD,165.00,https://www.boardgameatlas.com/api/price?clien...,Advent Games,US,au,2022,12,None,NaN
4,Chicken Caesar Game,0UXHZ6j2ez,2022-12-01 04:06:24,174.95,USD,0.00,https://www.boardgameatlas.com/api/price?clien...,Amazon,US,us,2022,12,etDV6aPHaG,NaN


In [89]:
atlas_prices_df.describe().toPandas()

,summary,game_name,atlas_price_id,price,currency,msrp,url,store_name,country,price_category,year,month,atlas_game_id,bgg_game_id
0,count,186791,186807,186807,186807,186807,186807,186806,186807,186807,186807,186807,62904,65315
1,mean,3104.5227272727275,None,35.18313232373548,None,44.194532324805856,None,None,None,None,2021.9999946468815,11.584191170566413,None,196534.2750822935
2,stddev,2300.8320485751515,None,40.77147556557906,None,1439.7071935266918,None,None,None,None,0.002313680716771292,0.8240806918473281,None,90583.78751244425
3,min,Bicycle Po-Ke-No Game,000yvkFTuZ,0.0,AUD,0.0,https://www.boardgameatlas.com/api/price?clien...,217 Comics Cards & Games,CA,au,2021,5,00ypZiDeXF,1
4,max,🔥 Ethnos Board Game / CMON / Brand New & Sealed 🔥,zzzKvCYeQi,2152.8,USD,449500.0,https://www.boardgameatlas.com/api/price?clien...,gameknight.ca,US,used,2022,12,zzmsZaTDjT,375438


#### Summary of Data

The prices table above indicates that:
- There are ~187k prices which all have an atlas_price_id, currency, msrp, url, store_name, country, and price category. Almost all rows have a game_name.
- However, only approximately 35% of atlas_game_id and bgg_game_id were able to be joined with the table based on the game name.
- From inspecting the sample of game_name in the above table, it can be seen that emojis and general descriptions may be present in the game_name. This would be the issue for joining on game_name.

Overall, the price fact table is limited based on the game_name description with how many rows can be joined with the board game detail dimensional tables. However, this still provides useful information for a large number of pricing data. A future step would involve a method to clean-up the game_name provided in the price data. This would not be trivial as the task isn't limited to removing emojis but also text such as 'Brand New & Sealed'. Something along the lines of parsing through the strings for an accepted set of board game names would likely be required.

In [90]:
atlas_prices_df.write.parquet(os.path.join(output_data, "prices.parquet"), partitionBy=["year", "month"], mode="overwrite")

### BGGLists Table

The fact table corresponding to `https://boardgamegeek.com/xmlapi/geeklist/` which provides GeekList information - saved to `/data/raw/bgg/lists.csv`.

In [91]:
# get filepath and read data
bgg_lists_data = os.path.join(input_data, "bgg/lists.csv")
bgg_lists_df = spark.read.csv(bgg_lists_data, header=True, inferSchema=True, multiLine=True)

In [92]:
bgg_lists_df.printSchema()

root
 |-- geeklist_id: string (nullable = true)
 |-- objtype: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user: string (nullable = true)
 |-- postdate: string (nullable = true)
 |-- bodytext: string (nullable = true)



In [94]:
# select columns and rename corresponding to the data model
bgg_lists_df = bgg_lists_df.withColumnRenamed("geeklist", "geeklist_id"
                                    ).withColumnRenamed("game_id", "bgg_game_id"
                                    ).withColumnRenamed("name", "game_name"
                                    ).withColumnRenamed("user", "bgg_user_name"
                                    ).withColumnRenamed("postdate", "post_datetime"
                                    ).withColumnRenamed("bodytext", "description")


In [95]:
# convert column types
bgg_lists_df = bgg_lists_df.withColumn("post_datetime", 
                                       to_timestamp(col("post_datetime"),format="EEE, dd MMM yyyy HH:mm:ss +SSSS")
                                        ).withColumn("geeklist_id", col("geeklist_id").cast(IntegerType())
                                        ).withColumn("bgg_game_id", col("bgg_game_id").cast(IntegerType()))

In [96]:
# add UID to Table and drop any missing geeklist_id or timestamp
bgg_lists_df = bgg_lists_df.withColumn("list_item_id", monotonically_increasing_id()
                              ).dropna(subset=["geeklist_id", "post_datetime"])

In [97]:
# add year and month columns for data partitioning for Parquet files
bgg_lists_df = bgg_lists_df.withColumn("year", year("post_datetime")).withColumn("month", month("post_datetime"))

Need to add the atlas_game_id to the fact table based on the game_name.

In [98]:
atlas_game_name = atlas_game_name.dropDuplicates(["game_name"])
bgg_lists_df = bgg_lists_df.join(atlas_game_name, "game_name", how='left')

In [99]:
bgg_lists_df.limit(5).toPandas()

,game_name,geeklist_id,objtype,bgg_game_id,bgg_user_name,post_datetime,description,list_item_id,year,month,atlas_game_id
0,Assembly,307798,boardgame,217353,evrenoz,2022-12-03 08:19:02,Status: Shortlisted\nFiles: Not downloaded\nCo...,0,2022,12,6jML0WjvPQ
1,Destination Earth,307798,boardgame,255468,evrenoz,2022-12-03 08:44:27,Status: Shortlisted\nFiles: Downloaded https:/...,1,2022,12,None
2,[Contest Ready] Creeping Crud (2018 Mint Tin D...,307798,None,2095892,evrenoz,2022-12-03 09:46:27,Status: Shortlisted\nFiles: Downloaded \nhttps...,2,2022,12,None
3,WordCraft,307798,boardgame,366463,evrenoz,2022-12-03 09:53:04,Status: Shortlisted\nFiles: Not available \nCo...,3,2022,12,None
4,Varuna,307798,boardgame,330664,evrenoz,2022-12-03 09:56:06,Status: Shortlisted\nFiles: Not available \nCo...,4,2022,12,None


In [100]:
bgg_lists_df.describe().toPandas()

,summary,game_name,geeklist_id,objtype,bgg_game_id,bgg_user_name,description,list_item_id,year,month,atlas_game_id
0,count,350842,350852,348433,350852,350852,281051,350852,350852,350852,243688
1,mean,Infinity,298986.55653950956,None,191288.6108843615,None,2.0134228187919463E99,242160.24634888786,2021.8843700477694,6.644208954202912,None
2,stddev,NaN,4262.71530020785,None,172050.40068223744,None,4.484103503261552E100,136581.69250412448,0.3197811092572016,3.2305308221227884,None
3,min,top 20 solo games 2022 edition,290003,boardgame,1,08thMS,\t\t\t\t\t\t\t\t\t\nName: \tUpTurn\t\t\t\t\t\t...,0,2021,1,00AI3sb84B
4,max,신묘한 사다리 (Mysterious Ladder),307798,videogameseries,2981144,zzzzz9,🦗🦗🦗\n\n[size=7]Those are crickets (meaning no ...,485696,2022,12,zzmsZaTDjT


#### Summary of Data

The lists table above indicates that:
- There are ~351k rows of data which all have a geeklist_id.
- Most rows >99.9% also have the bgg_game_id and game_name.
- Approximately 70% of the games were able to have the atlas_game_id joined. 

The 70% join of atlas_game_id makes sense as 85% of the bgg_game_id was able to be joined for the AtlasReviews Table and there is an order of magnitude higher number of bgg_game_id compared with atlas_game_id. This still provides the majority of the rows in the table with the ability to aggregate both BGG and Atlas game data from the dimensional tables.

In [101]:
bgg_lists_df.write.parquet(os.path.join(output_data, "lists.parquet"), partitionBy=["year", "month"], mode="overwrite")

### Time Table

The following dimensional table was created to allow easy extraction of datetime information without the need to run functions on the datetime data. The table is created to include all possible datetime information in the three fact tables such that the single dimensional table can be used.

In [102]:
# collect all datetimes
all_datetime_df = atlas_reviews_df.select(col("review_datetime").alias("datetime")
                                    ).union(
                                        atlas_prices_df.select(col("price_datetime").alias("datetime"))
                                    ).union(
                                        bgg_lists_df.select(col("post_datetime").alias("datetime"))
                                    ).dropDuplicates(["datetime"])

In [103]:
# extract columns of interest matching the data model
time_df = all_datetime_df.select(
    "datetime",
    minute("datetime").alias("minute"),
    hour("datetime").alias("hour"),
    dayofmonth("datetime").alias("day"),
    weekofyear("datetime").alias("week"),
    month("datetime").alias("month"),
    year("datetime").alias("year"),
    dayofweek("datetime").alias("weekday")
)

In [104]:
time_df.limit(5).toPandas()

,datetime,minute,hour,day,week,month,year,weekday
0,2020-03-24 18:54:40,54,18,24,13,3,2020,3
1,2020-01-04 03:31:28,31,3,4,1,1,2020,7
2,2021-03-16 11:07:31,7,11,16,11,3,2021,3
3,2020-04-01 19:41:44,41,19,1,14,4,2020,4
4,2020-07-24 05:20:49,20,5,24,30,7,2020,6


In [105]:
time_df.write.parquet(os.path.join(output_data, "time.parquet"), partitionBy=["year", "month"], mode="overwrite")

## Quality Checks
The following quality checks are performed for each Table. These checks will be included prior to writing data to Parquet format:
- confirm that the number of columns matches the expected number
- confirm that the DataFrame contains data
- confirm that any columns required as Not Null do not contain any missing values

In [110]:
check_tables = {
    "BGGDetails Table": 
    {"file": "bgg_games.parquet",
     "columns": ["bgg_game_id"],
     "count": 18},
    "AtlasUsers Table": 
    {"file": "users.parquet",
     "columns": ["user_id", "user_name"],
     "count": 11},
    "AtlasDetails Table": 
    {"file": "atlas_games.parquet",
     "columns": ["atlas_game_id"],
     "count": 18},
    "AtlasReviews Table":
    {"file": "reviews.parquet",
     "columns": ["review_id", "review_datetime"],
     "count": 10},
    "AtlasPrices Table":
    {"file": "prices.parquet",
     "columns": ["atlas_price_id", "price_datetime"],
     "count": 14},
    "BGGLists Table":
    {"file": "lists.parquet",
     "columns": ["list_item_id", "post_datetime"],
     "count": 11},
    "Time Table":
    {"file": "time.parquet",
     "columns": ["datetime"],
     "count": 8},
    
}

In [111]:
for table, params in check_tables.items():
    file = params["file"]
    data = spark.read.parquet(os.path.join(output_data, file))
    col_count = len(data.columns)
    row_count = data.count()
    
    if row_count == 0:
        print(f"Row check failed: {table} has no rows")
    else:
        print(f"Row check passed: {table} has {row_count} rows")
        
    expect_col = params["count"]
    if col_count != expect_col:
        print(f"Column check failed: {table} has {col_count} instead of {expect_col} columns")
    else:
        print(f"Column check passed: {table} has {col_count} columns")
    
    for check in params["columns"]:
        check_null = data.filter(col(check).isNull()).count()
        if check_null > 0:
            print(f"Null check failed for {check} column in {table}, there are {check_null} values")
        else:
            print(f"Null check passed for {check} column in {table}")

Row check passed: BGGDetails Table has 342570 rows
Column check passed: BGGDetails Table has 18 columns
Null check passed for bgg_game_id column in BGGDetails Table
Row check passed: AtlasUsers Table has 5423 rows
Column check passed: AtlasUsers Table has 11 columns
Null check passed for user_id column in AtlasUsers Table
Null check passed for user_name column in AtlasUsers Table
Row check passed: AtlasDetails Table has 23500 rows
Column check passed: AtlasDetails Table has 18 columns
Null check passed for atlas_game_id column in AtlasDetails Table
Row check passed: AtlasReviews Table has 226871 rows
Column check passed: AtlasReviews Table has 10 columns
Null check passed for review_id column in AtlasReviews Table
Null check passed for review_datetime column in AtlasReviews Table
Row check passed: AtlasPrices Table has 186807 rows
Column check passed: AtlasPrices Table has 14 columns
Null check passed for atlas_price_id column in AtlasPrices Table
Null check passed for price_datetime c

## Summary

The EDA and associated data wrangling to create the pipelines is completed in this notebook resulting in the three fact tables using the same associated dimensional tables. The AtlasReviews and BGGLists tables were able to successfully aggregate both Atlas and BGG board game data for the majority of rows. However, the AtlasPrices tables were able to join with less board game data as discussed due to the game_name provided with the prices.

Overall, the data models provides rich datasets that could be used to support an application understanding any or all of game prices, reviews, and popularity.